In [1]:
import pandas as pd

In [2]:
url = 'https://raw.githubusercontent.com/mattharrison/datasets/master/data/siena2018-pres.csv'
df = pd.read_csv(url)

In [7]:
def tweak_sienna(df):
    def int64_uint8(df_):
        cols = df_.select_dtypes('int64')
        return (df_
                .astype({col: 'uint8' for col in cols}))
    
    return (df
            .rename(columns={'Seq.': 'Seq'})
            .rename(columns={k:v.replace(' ', '_') for k, v in 
                    {'Bg ': ' Background ',
                     'PL ': ' Party leadership ', 
                     'CAb ': ' Communication ability ' ,
                     'RC ': ' Relations with Congress ', 
                     'CAp ': ' Court appointments ' ,
                     'HE ': ' Handling of economy ', 
                     'L ': 'Luck ' ,
                     'AC ': ' Ability to compromise ',
                     'WR ': ' Willing to take risks ' ,
                     'EAp ': ' Executive appointments ', 
                     'OA ': ' Overall ability ' ,
                     'Im ': ' Imagination ', 
                     'DA ': ' Domestic accomplishments ' ,
                     'Int ': ' Integrity ', 
                     'EAb ': ' Executive ability ' ,
                     'FPA ': ' Foreign policy accomplishments ' ,
                     'LA ': ' Leadership ability ',
                     'IQ ': ' Intelligence ', 
                     'AM ': ' Avoid crucial mistakes ' ,
                     'EV ': " Experts ' view ", 
                     'O ': ' Overall '}.items()})
           .astype({'Party': 'category'})
           .pipe(int64_uint8)
           .assign(Average_rank=lambda df_:(df_.select_dtypes('uint8')
                        .sum(axis=1).rank(method='dense').astype('uint8')),
                   Quartile=lambda df_:pd.cut(df_.Average_rank, 4, 
                                              labels='1st 2nd 3rd 4th'.split())))

In [4]:
df = tweak_sienna(df).drop(columns=['Unnamed: 0'])

In [5]:
df.head()

,Seq,President,Party,Bg,Im,Int,IQ,L,WR,AC,...,CAp,HE,EAp,DA,FPA,AM,EV,O,Average_rank,Quartile
0,1,George Washington,Independent,7,7,1,10,1,6,2,...,1,1,1,2,2,1,2,1,1,1st
1,2,John Adams,Federalist,3,13,4,4,24,14,31,...,4,13,15,19,13,16,10,14,12,2nd
2,3,Thomas Jefferson,Democratic-Republican,2,2,14,1,8,5,14,...,7,20,4,6,9,7,5,5,5,1st
3,4,James Madison,Democratic-Republican,4,6,7,3,16,15,6,...,6,14,7,11,19,11,8,7,6,1st
4,5,James Monroe,Democratic-Republican,9,14,11,18,6,16,7,...,11,9,9,10,5,6,9,8,7,1st


## looping iterrows

In [6]:
for idx, row in df.iterrows():
    print(idx, row.President, row.Party)
    break

0 George Washington Independent


## aggregations

In [7]:
scores = (df.loc[:, 'Bg':'Average_rank'].sum(axis=1)/(df.shape[1]-4))

In [8]:
scores

0      3.681818
1     14.409091
2      6.545455
3      9.590909
4     10.409091
5     17.136364
6     19.590909
7     25.681818
8     36.909091
9     34.409091
10    13.272727
11    29.500000
12    37.454545
13    39.363636
14    42.000000
15     4.000000
16    42.272727
17    24.227273
18    30.090909
19    27.181818
20    31.454545
21    22.181818
22    32.818182
23    19.727273
24     5.227273
25    21.272727
26    13.636364
27    38.818182
28    29.954545
29    31.954545
30     3.954545
31    11.818182
32     9.318182
33    12.772727
34    15.272727
35    26.909091
36    26.000000
37    26.909091
38    14.545455
39    20.863636
40    14.636364
41    30.363636
42    15.863636
43    39.772727
dtype: float64

## multiple aggregations

In [9]:
df.loc[:, 'Average_rank'].agg(['count', 'sum'])

count     44
sum      990
Name: Average_rank, dtype: int64

In [10]:
#aggregations based on each party
df.groupby('Party').agg(['count','sum']).Average_rank

,count,sum
Party,,
Democratic,15,318.0
Democratic-Republican,4,35.0
Federalist,1,12.0
Independent,2,38.0
Republican,19,480.0
Whig,3,107.0


In [14]:
# or we can also do above command as 
df[['Party', 'Average_rank']].groupby(['Party']).agg(['count', 'sum'])

Average_rank       
                             count    sum
Party                                    
Democratic                      15  318.0
Democratic-Republican            4   35.0
Federalist                       1   12.0
Independent                      2   38.0
Republican                      19  480.0
Whig                             3  107.0

In [15]:
#multiple aggregation function
df.groupby('Party').agg({'Average_rank': ['max', 'min'],
                         'O': ['mean', 'max']})[['Average_rank', 'O']]

Average_rank              O    
                               max min       mean max
Party                                                
Democratic                      44   3  20.800000  44
Democratic-Republican           17   5   9.500000  18
Federalist                      12  12  14.000000  14
Independent                     37   1  19.000000  37
Republican                      42   2  25.315789  42
Whig                            39  30  35.666667  39

In [18]:
# can also using the same code as below
df[['Party','Average_rank', 'O']].groupby(['Party']).agg({'Average_rank': ['max', 'min'],
                                                 'O': ['mean', 'max']})

Average_rank              O    
                               max min       mean max
Party                                                
Democratic                      44   3  20.800000  44
Democratic-Republican           17   5   9.500000  18
Federalist                      12  12  14.000000  14
Independent                     37   1  19.000000  37
Republican                      42   2  25.315789  42
Whig                            39  30  35.666667  39

### Apply method, combine with lambda

In [23]:
def func(party, overall):
    if party == 'Republican' and overall > 26:
        return 'good'
    elif party == 'Republican' and overall < 26:
        return 'not so good'
    else:
        return 'trash'

In [31]:
#NOTE: if we want access to column value use axis=1 or axis='columns'
df['label'] = df.apply(lambda df: func(df.Party, df.O), axis='columns')

In [32]:
df.head()

,Seq,President,Party,Bg,Im,Int,IQ,L,WR,AC,...,HE,EAp,DA,FPA,AM,EV,O,Average_rank,Quartile,label
0,1,George Washington,Independent,7,7,1,10,1,6,2,...,1,1,2,2,1,2,1,1,1st,trash
1,2,John Adams,Federalist,3,13,4,4,24,14,31,...,13,15,19,13,16,10,14,12,2nd,trash
2,3,Thomas Jefferson,Democratic-Republican,2,2,14,1,8,5,14,...,20,4,6,9,7,5,5,5,1st,trash
3,4,James Madison,Democratic-Republican,4,6,7,3,16,15,6,...,14,7,11,19,11,8,7,6,1st,trash
4,5,James Monroe,Democratic-Republican,9,14,11,18,6,16,7,...,9,9,10,5,6,9,8,7,1st,trash
